# __Auto-ML을 사용하여 분류 모델 만들기__

**분류 작업 이해하기** 
--- 
👉 분류 작업은 목표값(Target)이 속한 범주(Category 또는 Class)를 예측하기 위해 사용하는 머신러닝(기계학습/Machine Learning)의 한 형태입니다. 예를 들어, 남성 또는 여성을 분류하는 이진 분류와 동물의 종(개, 고양이, 토끼 등)을 예측하는 다중 분류 모두 분류 작업에 포함됩니다.

**본 튜토리얼에서는** 
--- 
👉 대표적인 머신러닝 경진대회 플랫폼인 캐글의 입문자를 위한 Titanic: Machine Learning from Disaster 데이터 세트를 사용하여 생존자 예측 분류 모델을 만듭니다. 이 대회의 목표는 아래와 같습니다. (참고로, 해당 대회의 데이터는 1912년 4월 15일 실제 타이타닉 사건 때, 탑승했었던 승객들 명단입니다.)


## __0. 데이터 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql)
에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.   

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN={발급받은 개인 토큰}

In [2]:
%%thanosql
COPY titanic_train FROM "tutorial_data/titanic_data/titanic_train.csv"

Success


In [3]:
%%thanosql
COPY titanic_test FROM "tutorial_data/titanic_data/titanic_test.csv"

Success


## __1. 데이터 세트 확인__

생존자 예측 분류 모델을 만들기 위해 우리는 ThanoSQL DB에 저장되어 있는 titanic_train 테이블을 사용합니다. 아래의 쿼리문을 실행하면서 테이블 내용을 확인합니다.

In [4]:
%%thanosql
SELECT * 
FROM titanic_test 
LIMIT 5 

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**데이터 이해하기**   
-  passengerid  : 탑승승객 아이디  
-  survived  : 탑승승객  
-  pclass  : 탑승승객 티켓 등급  
-  name  : 탑승승객 이름  
-  sex  : 탑승승객 성별  
-  age  : 탑승승객 나이  
-  sibsp  : 탑승한 형제 자매 또는 배우자 수   
-  parch  : 탑승한 부모 또는 자녀의 수  
-  ticket  : 티켓 번호  
-  fare  : 요금  
-  cabin  : 선실  
-  embarked  : 승선지 또는 항구   

이번 튜토리얼에서는 추가적인 쿼리문을 이용한 데이터 전처리가 필요한  name ,  ticket ,  cabin 컬럼을 제외하고 모델 학습을 진행하겠습니다.

## __2. 분류 모델 생성__

이전 단계에서 확인한 titanic_train 데이터를 사용하여 생존자 예측 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행시켜 titanic_classification 이름의 모델을 만들어 봅니다.

In [7]:
%%thanosql
BUILD MODEL tutorial_automl_classification 
USING AutomlClassifier 
OPTIONS (
    target='survived', 
    impute_type='iterative',  
    features_to_drop=["name", 'ticket', 'passengerid', 'cabin'],
    time_left_for_this_task = 300
    ) 
AS 
SELECT * 
FROM titanic_train

Success


__쿼리 세부 정보__ 

- "__BUILD MODEL__" 쿼리 구문을 사용하여 <mark style="background-color:#E9D7FD ">titanic_automl_classification</mark>이라는 모델을 만들고 학습시킵니다.  
- "__OPTIONS__"의 "target"에는 분류 예측 모델에 목표값이 되는 컬럼(Column)의 이름을 적어줍니다. "impute_type"의 경우에는 데이터 세트의 빈 값에 대한 처리를 의미합니다. "features_to_drop"은 학습에 이용하지 않을 데이터를 적어주면 머신러닝 모델 생성을 진행할 수 있습니다.  

## __3. 생성된 모델 평가__

아래의 쿼리문을 실행하여 이전 단계에서 만든 예측 모델의 성능을 평가합니다. 

In [8]:
%%thanosql 
EVALUATE USING tutorial_automl_classification 
OPTIONS (
    target = 'survived'
    )
AS 
SELECT * 
FROM titanic_train

,metrics,value
0,accuracy,0.897868
1,ROCAUC,0.895374
2,Recall,0.886154
3,Precision,0.842105
4,f1-score,0.863568
5,Kappa,0.782039
6,MCC,0.782688


__ 쿼리 세부 정보__
- "__EVALUATE USING__" 쿼리 구문을 사용하여 구축한 titanic_automl_classification이라는 모델을 평가합니다. 
- "__OPTIONS__"의 "target"에는 분류 예측 모델에 목표값이 되는 컬럼의 이름(survived)을 적어줍니다.

## __4. 생성된 모델을 사용하여 생존자 예측__

이전 단계에서 생성한 생존자 예측 모델을 사용해 탑승 승객 정보에 따른 생존 여부를 예측해 봅니다. 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, titanic_test)를 사용합니다.

In [9]:
%%thanosql 
PREDICT USING tutorial_automl_classification
AS 
SELECT * 
FROM titanic_test

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,predicted
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,None,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,None,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,None,S,0


__쿼리 세부 정보__ 

- "PREDICT USING" 쿼리 구문을 사용하여 이전 단계에서 만든 tutorial_automl_classification 모델을 예측에 사용합니다. "PREDICT"의 경우 생성된 모델의 절차를 따르기 때문에 특별한 처리가 필요없습니다.